# FEATURE EXTRACTION - DEMO

In [5]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#project variables
demopath=os.getcwd()


In [6]:
#define features path
dreamspath=demopath+"/DREAMS"
featurespath=dreamspath+"/features"
if not os.path.isdir(featurespath):
    os.mkdir(featurespath)
        
windowDurations=[0.5,1,1.5,2]

## define a fixed samplerate

In [7]:
samplerate=200

## Load data

In [8]:
signals, annotations, signalsMetadata = loadDREAMSSpindlesDemo(demopath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [9]:
signalsMetadata

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS
5,excerpt6.txt,CZ-A1,0006,1800,200,DREAMS
6,excerpt7.txt,CZ-A1,0007,1800,200,DREAMS
7,excerpt8.txt,CZ-A1,0008,1800,200,DREAMS


In [10]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)

In [11]:
annotations.head()

,index,startTime,duration,channel,subjectId,labelerId,type,samplerate,stopTime,startInd,stopInd
0,0,282.24,0.72,C3-A1,0001,0001,spindle,200,282.96,56448,56592
1,1,311.72,1.54,C3-A1,0001,0001,spindle,200,313.26,62344,62652
2,2,340.28,0.72,C3-A1,0001,0001,spindle,200,341.00,68056,68200
3,3,366.83,0.65,C3-A1,0001,0001,spindle,200,367.48,73366,73496
4,4,373.74,0.50,C3-A1,0001,0001,spindle,200,374.24,74748,74848


## Preprocess

broadband in 0-40Hz

In [12]:
def preprocessVector(vector,samplerate):
    
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Robust Z-score
    from sklearn.preprocessing import RobustScaler
    vector_preprocessed=RobustScaler().fit_transform(vector.reshape(-1, 1) )
    return vector_preprocessed. flatten()

In [13]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

## Extract features

### folder estructure

In [14]:
fspath=featurespath+'/'+str(samplerate)+'fs/'
if not os.path.isdir(fspath):
    os.mkdir(fspath)
for window in windowDurations:
    windowPath=fspath+str(window)+'win'
    if not os.path.isdir(windowPath):
        os.mkdir(windowPath)
    for subject in signalsMetadata.subjectId:
        subjectPath=windowPath+'/'+subject
        if not os.path.isdir(subjectPath):
            os.mkdir(subjectPath)

### band definition

In [15]:
bands={
    'delta1':[0.1,2],
    'delta2':[2,4],
    'theta':[4,8],
    'alpha':[8,13],
    'sigma':[11,16],
    'beta1':[13,19],
    'beta2':[19,30]
    }

### computation

In [16]:
for window in windowDurations:
    windowPath=featurespath+'/'+str(samplerate)+'fs/'+str(window)
    for ind, row in signalsMetadata.iterrows():
        subject=row.subjectId
        subjectPath=windowPath+'/'+subject
        signal=signals[subject]
        #need to define the time vector for each signal:
        timepoints=np.arange(len(signal))/samplerate
        
        #6. Hjort Activity
        characteristic='hjortActivity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortActivity(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortActivity(filtered,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #7. Hjort Mobility
        characteristic='hjortMobility'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortMobility(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortMobility(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #8. Hjort Complexity
        characteristic='hjortComplexity'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=hjortComplexity(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=hjortComplexity(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        #9. PetrosianFractalDimension
        characteristic='petrosian'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=petrosianFractalDimension(signal,timepoints,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        for bandName, band in bands.items():
            filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
            if not os.path.exists(filepath):
                filtered=filterBand(signal,band,samplerate)
                aux=petrosianFractalDimension(filtered,timepoints,window,samplerate)
                saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
        
        #12. sigma index
        characteristic='sigmaIndex'
        bandName='broadband'
        filepath=getFilepath(window,subject,characteristic,bandName,samplerate,featurespath)
        if not os.path.exists(filepath):
            aux=sigmaindex(signal,window,samplerate)
            saveFeature(aux,window,subject,characteristic,bandName,samplerate,featurespath)
            

saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_broadband.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta1.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_delta2.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_theta.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_alpha.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_sigma.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/features/200fs/0.5win/0001/0.5_0001_hjortActivity_beta1.fd
saving feature: C:\Users\GIB\Documents\GitHub\SpinCo\SpinCo\demo/DREAMS/featu